# Create necessary classes

## General classes

In [252]:
# package(s) related to the simulation
import simpy

# package(s) related to time, space and id
import datetime
import json
import geojson
import shapely.geometry
import uuid

In [253]:
class Identifiable(object):
    """Something that has a name and id
    
    env: a simpy Environment
    name: a name
    id: a unique id generated with uuid"""
    
    def __init__(self, env, name, id=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.env = env
        self.name = name
        # generate some id, in this case based on m
        self.id = id if id else str(uuid.uuid1())
                

In [254]:
class Location(object):
    """Something with a geometry (geojson format)
    
    geometry: can be a point as well as a polygon"""
    
    def __init__(self, geometry, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.geometry = geometry


In [255]:
class Container(object):
    """Container class
    
    capacity: amount the container can hold
    level: amount the container holds
    container: a simpy object that can hold stuff
    total_requested: a counter needed to prevent over-handling"""

    def __init__(self, capacity, level=0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.container = simpy.Container(env, capacity, init=level)
        self.total_requested = 0


In [256]:
class Move(object):
    """Move class
    
    origin: origin of trip
    destination: destination of trip
    v_empty: speed empty [m/s]
    v_full: speed full [m/s]
    resource: a simpy resource that can be requested"""
    
    def __init__(self, 
                 origin, destination,
                 v_empty=1.5, v_full=1, 
                 nr_resources=1,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.origin = origin
        self.destination = destination
        self.v_empty = v_empty 
        self.v_full = v_full 
        self.resource = simpy.Resource(env, capacity=nr_resources)

    def execute_move(self, origin, destination):
        """determine distance between origin and destination, and
        yield the time it takes to travel it"""
        origin = shapely.geometry.asShape(self.origin.geometry)
        destination = shapely.geometry.asShape(self.destination.geometry)
        distance = origin.distance(destination) 

        if self.container.level == self.container.capacity:
            yield self.env.timeout(distance / self.v_full)
            print('  sailing full: ' + str(self.v_full) + ' m/s')
 
        elif self.container.level == 0:
            yield self.env.timeout(distance / self.v_empty)
            print('  sailing empty: ' + str(self.v_empty) + ' m/s')
 

In [257]:
class Process(object):
    """Process class
    
    resource: a simpy resource that can be requested
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    rate: rate with which quantity can be processed [amount/s]
    amount: amount to process"""

    def __init__(self, 
                 nr_resources,
                 origin, destination, rate, amount=0,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.resource = simpy.Resource(env, capacity=nr_resources)
        self.origin=origin
        self.destination=destination
        self.rate=rate
        self.amount=amount
        
    def execute_process(self, origin, destination, amount):
        """get amount from origin container,
        put amount in destination continater, and
        yield the time it takes to process it"""
        origin.container.get(amount)
        destination.container.put(amount)
        yield self.env.timeout(amount / self.rate)
        

In [258]:
class Log(object):
    """Log class
    
    log: log message [format: 'start activity' or 'stop activity']
    t: timestamp
    value: a value can be logged as well"""
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.log = []
        self.t = []
        self.value = []
        
    def log_entry(self, log, t, value):
        """Log"""
        self.log.append(log)
        self.t.append(t)
        self.value.append(value)
        

## Constructed classes

In [259]:
class Site(Identifiable, Location, Log, Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [260]:
class Transport_Resource(Identifiable, Location, Log, Container, Move):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [261]:
class Transport_Processing_Resource(Identifiable, Location, Log, Container, Move, Process):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [262]:
class Processing_Resource(Identifiable, Location, Log, Process):
    def __init__(self, 
                 *args, **kwargs):
        super().__init__(*args, **kwargs)


## Activities

In [263]:
class Installation(Identifiable):
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. 
    
    This class deals with transport and installation/placement of discrete and continuous objects.
    
    condition: expression that states when to initiate or to suspend activity
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    loader: gets amount from origin Contaner and puts it into mover Container
    mover: moves amount in Container from origin to destination
    unloader: gets amount from mover Contaner and puts it into destination Container"""

    def __init__(self, 
                 condition,
                 origin, destination,  
                 loader, mover, unloader,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        
        self.condition = condition
        
        self.origin = origin
        self.destination = destination
        
        self.loader = loader
        self.mover = mover
        self.unloader = unloader
        
        self.standing_by_proc = env.process(
            self.standing_by(env, 
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_proc = env.process(
            self.installation_process_control(env,
                             condition,
                             origin, destination,
                             loader, mover, unloader))

        self.installation_reactivate = env.event()

    def standing_by(self, env, condition,
                          origin, destination,
                          loader, mover, unloader):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(env.now) + ' ' + self.name + ' to ' + destination.name + ' suspended')
                shown = True
            yield env.timeout(1) # step 1 time unit ahead

        print('T=' + '{:06.2f}'.format(env.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = env.event()
 
    def installation_process_control(self, env, condition,
                                           origin, destination,
                                           loader, mover, unloader):
        """Installation process control"""  
        while not eval(condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(env.now) + ' '+ self.name + ' to ' + destination.name + ' started')
        while eval(condition):
            yield from self.installation_process(env, condition,
                                                 origin, destination,
                                                 loader, mover, unloader)
        
    def installation_process(self, env, condition,
                                   origin, destination,
                                   loader, mover, unloader):
        """Installation process"""
        amount = min(
            mover.container.capacity - mover.container.level,
            destination.container.capacity - destination.container.level)

        if amount>0:
            print('amount>0')
            # request access to the load_resource
            with loader.resource.request() as my_load_resource_turn:
                yield my_load_resource_turn

                destination.total_requested += amount
                origin.total_requested += amount

                mover.log_entry('loading start', self.env.now, mover.container.level)
                print('Loading:')
                yield from loader.execute_process(origin, mover, amount)
                mover.log_entry('loading stop', self.env.now, mover.container.level)
 
                loader.resource.release(my_load_resource_turn)

                print('  ' + origin.name + ' contains: ' + str(origin.container.level))
                print('  ' + mover.name + ' contains: ' + str(mover.container.level))
                print('  ' + destination.name + ' contains: ' + str(destination.container.level))
 
            # request access to the transport_resource
            with mover.resource.request() as my_mover_turn:
                yield my_mover_turn

                mover.log_entry('sailing full start', self.env.now, mover.container.level)
                yield from mover.execute_move(origin, destination)
                mover.log_entry('sailing full stop', self.env.now, mover.container.level)

            # request access to the placement_resource
            with unloader.resource.request() as my_unloader_turn:
                yield my_unloader_turn

                mover.log_entry('unloading start', self.env.now, mover.container.level)
                print('Unloading:')
                yield from loader.execute_process(mover, destination, amount)
                mover.log_entry('unloading stop', self.env.now, mover.container.level)

                unloader.resource.release(my_unloader_turn)

                print('  ' + origin.name + ' contains: ' + str(origin.container.level))
                print('  ' + mover.name + ' contains: ' + str(mover.container.level))
                print('  ' + destination.name + ' contains: ' + str(destination.container.level))

            # request access to the transport_resource
            with mover.resource.request() as my_mover_turn:
                yield my_mover_turn

                mover.log_entry('sailing full start', self.env.now, mover.container.level)
                yield from mover.execute_move(destination, origin)
                mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                mover.resource.release(my_mover_turn)


# Start case

In [264]:
import simpy

In [265]:
# *** Create a project environment
env = simpy.Environment()

In [266]:
# *** Define sites

# Den Oever
# <longitude>5.070195628786471</longitude>
# <latitude>52.93917167503315</latitude>

# Breezanddijk
# <longitude>5.197016484858931</longitude>
# <latitude>53.0229621352376</latitude>

# Kornwerderzand
# <longitude>5.310252030674185</longitude>
# <latitude>53.06330503232375</latitude>

# Den Oever haven
# <longitude>5.019298185633251</longitude>
# <latitude>52.94239823421129</latitude>

# sites in database (nr indicates km's from Den Oever haven)
data_site_00 = {"env": env,
                "name": "KP00", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "capacity": 15000, "level": 15000}
data_site_01 = {"env": env,
                "name": "KP01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 5000, "level": 0}
data_site_15 = {"env": env,
                "name": "KP15", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "capacity": 5000, "level": 0}
data_site_32 = {"env": env,
                "name": "KP32", "geometry": geojson.Point([53.06330503232375, 5.310252030674185]),
                "capacity": 5000, "level": 0}

# create site objects
site_00 = Site(**data_site_00)
site_01 = Site(**data_site_01)
site_15 = Site(**data_site_15)
site_32 = Site(**data_site_32)

# print the outputs
print(site_00.__dict__)
print(site_01.__dict__)
print(site_15.__dict__)
print(site_32.__dict__)

{'container': <simpy.resources.container.Container object at 0x000002214CFAA208>, 'total_requested': 0, 'log': [], 't': [], 'value': [], 'geometry': {"coordinates": [52.94239823421129, 5.019298185633251], "type": "Point"}, 'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'KP00', 'id': '2718059e-893f-11e8-8c90-60f677ba2ce0'}
{'container': <simpy.resources.container.Container object at 0x000002214CFAA438>, 'total_requested': 0, 'log': [], 't': [], 'value': [], 'geometry': {"coordinates": [52.93917167503315, 5.070195628786471], "type": "Point"}, 'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'KP01', 'id': '2718059f-893f-11e8-b1fd-60f677ba2ce0'}
{'container': <simpy.resources.container.Container object at 0x000002214CFAADD8>, 'total_requested': 0, 'log': [], 't': [], 'value': [], 'geometry': {"coordinates": [53.0229621352376, 5.197016484858931], "type": "Point"}, 'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'KP15', 'id': 

In [267]:
# *** Define fleet

# sites in database (nr indicates km's from Den Oever haven)
# - processing resources
data_gantry_crane = {"env": env,
                "name": "Gantry crane", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "origin": site_00, "destination": site_01, "rate": 0.10, "nr_resources": 1}
data_installation_crane = {"env": env,
                "name": "Installation crane", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "origin": site_00, "destination": site_01, "rate": 0.05, "nr_resources": 1}

# - transport resources
data_transport_barge_01 = {"env": env,
                "name": "Transport barge 01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1, "origin": site_00, "destination": site_01}
data_transport_barge_02 = {"env": env,
                "name": "Transport barge 02", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1, "origin": site_00, "destination": site_01}

# create site objects
# - processing resources
gantry_crane = Processing_Resource(**data_gantry_crane)
installation_crane = Processing_Resource(**data_installation_crane)

# - transport resources
transport_barge_01 = Transport_Resource(**data_transport_barge_01)
transport_barge_02 = Transport_Resource(**data_transport_barge_02)

# print the outputs
print(gantry_crane.__dict__)
print(transport_barge_01.__dict__)
print(transport_barge_02.__dict__)
print(installation_crane.__dict__)


{'resource': <simpy.resources.resource.Resource object at 0x000002214CFED470>, 'origin': <__main__.Site object at 0x000002214CFAAE48>, 'destination': <__main__.Site object at 0x000002214CFC2B00>, 'rate': 0.1, 'amount': 0, 'log': [], 't': [], 'value': [], 'geometry': {"coordinates": [52.94239823421129, 5.019298185633251], "type": "Point"}, 'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'Gantry crane', 'id': '27204308-893f-11e8-9a2e-60f677ba2ce0'}
{'origin': <__main__.Site object at 0x000002214CFAAE48>, 'destination': <__main__.Site object at 0x000002214CFC2B00>, 'v_empty': 1.5, 'v_full': 1, 'resource': <simpy.resources.resource.Resource object at 0x000002214CFED390>, 'container': <simpy.resources.container.Container object at 0x000002214CFED7F0>, 'total_requested': 0, 'log': [], 't': [], 'value': [], 'geometry': {"coordinates": [52.93917167503315, 5.070195628786471], "type": "Point"}, 'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'Transport

In [268]:
# *** Define activties

# activities in database
data_act_1 = {"env": env,
              "name": "Block placement",
              "origin": site_00, "destination": site_01,
              "loader": gantry_crane, "mover": transport_barge_01, "unloader": installation_crane,
              "condition": '''site_01.container.level<5000'''}
data_act_2 = {"env": env,
              "name": "Block placement",
              "origin": site_00, "destination": site_15,
              "loader": gantry_crane, "mover": transport_barge_02, "unloader": installation_crane,
              "condition": '''site_15.container.level<5000 and site_01.container.level>=3000'''}

# create Activity objects
act_1 = Installation(**data_act_1)
act_2 = Installation(**data_act_2)

# print the outputs
print(act_1.__dict__)
print(act_2.__dict__)


{'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'Block placement', 'id': '27254c1a-893f-11e8-8ad6-60f677ba2ce0', 'condition': 'site_01.container.level<5000', 'origin': <__main__.Site object at 0x000002214CFAAE48>, 'destination': <__main__.Site object at 0x000002214CFC2B00>, 'loader': <__main__.Processing_Resource object at 0x000002214CFED208>, 'mover': <__main__.Transport_Resource object at 0x000002214CFED128>, 'unloader': <__main__.Processing_Resource object at 0x000002214CFED198>, 'standing_by_proc': <Process(standing_by) object at 0x2214cfb83c8>, 'installation_proc': <Process(installation_process_control) object at 0x2214cfb8080>, 'installation_reactivate': <Event() object at 0x2214cfb8400>}
{'env': <simpy.core.Environment object at 0x000002214CF81C88>, 'name': 'Block placement', 'id': '27257324-893f-11e8-9bf8-60f677ba2ce0', 'condition': 'site_15.container.level<5000 and site_01.container.level>=3000', 'origin': <__main__.Site object at 0x000002214CFAAE48>, 'd

In [269]:
#*** Run the project
env.run()

T=000.00 Condition: site_01.container.level<5000 is satisfied
T=000.00 Block placement to KP01 started
amount>0
T=000.00 Block placement to KP15 suspended
Loading:
  KP00 contains: 14000
  Transport barge 01 contains: 1000
  KP01 contains: 0
  sailing full: 1 m/s
Unloading:
  KP00 contains: 14000
  Transport barge 01 contains: 0
  KP01 contains: 1000
  sailing empty: 1.5 m/s
amount>0
Loading:
  KP00 contains: 13000
  Transport barge 01 contains: 1000
  KP01 contains: 1000
  sailing full: 1 m/s
Unloading:
  KP00 contains: 13000
  Transport barge 01 contains: 0
  KP01 contains: 2000
  sailing empty: 1.5 m/s
amount>0
Loading:
  KP00 contains: 12000
  Transport barge 01 contains: 1000
  KP01 contains: 2000
  sailing full: 1 m/s
Unloading:
T=50001.00 Condition: site_15.container.level<5000 and site_01.container.level>=3000 is satisfied
T=50001.00 Block placement to KP15 started
amount>0
Loading:
  KP00 contains: 11000
  Transport barge 01 contains: 0
  KP01 contains: 3000
  sailing empty: 1

# sandbox

In [270]:
# option to create a range of sites between two points
start = [52.93917167503315, 5.070195628786471] # Den Oever
stop = [53.0229621352376,  5.197016484858931]  # Breezanddijk
nums = 50

In [271]:
lats = np.linspace(start[0], start[1], num=nums)
lons = np.linspace(stop[0],  stop[1],  num=nums)

for i in range(nums):
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f'), "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": 5000, "level": 0}
    vars()['Site_' + "KP" + format(i,'02.0f')] = Site(**data_site)
    


In [272]:
Site_KP00.__dict__

{'container': <simpy.resources.container.Container at 0x2214cfc2438>,
 'env': <simpy.core.Environment at 0x2214cf81c88>,
 'geometry': {"coordinates": [52.93917167503315, 53.0229621352376], "type": "Point"},
 'id': '27afce18-893f-11e8-85b8-60f677ba2ce0',
 'log': [],
 'name': 'KP00',
 't': [],
 'total_requested': 0,
 'value': []}

In [273]:
lats[1]

51.96225379653832

In [274]:
import numpy as np

print(np.linspace(5.070195628786471, 5.197016484858931, num=50))
print(np.linspace(52.93917167503315, 53.0229621352376, num=50))

# Den Oever
# <longitude>5.070195628786471</longitude>
# <latitude>52.93917167503315</latitude>

# Breezanddijk
# <longitude>5.197016484858931</longitude>
# <latitude>53.0229621352376</latitude>

[5.07019563 5.07278381 5.07537199 5.07796017 5.08054835 5.08313653
 5.08572471 5.08831289 5.09090107 5.09348926 5.09607744 5.09866562
 5.1012538  5.10384198 5.10643016 5.10901834 5.11160652 5.1141947
 5.11678288 5.11937106 5.12195924 5.12454742 5.1271356  5.12972379
 5.13231197 5.13490015 5.13748833 5.14007651 5.14266469 5.14525287
 5.14784105 5.15042923 5.15301741 5.15560559 5.15819377 5.16078195
 5.16337014 5.16595832 5.1685465  5.17113468 5.17372286 5.17631104
 5.17889922 5.1814874  5.18407558 5.18666376 5.18925194 5.19184012
 5.1944283  5.19701648]
[52.93917168 52.94088168 52.94259169 52.9443017  52.94601171 52.94772172
 52.94943173 52.95114174 52.95285175 52.95456176 52.95627177 52.95798178
 52.95969179 52.9614018  52.96311181 52.96482182 52.96653183 52.96824183
 52.96995184 52.97166185 52.97337186 52.97508187 52.97679188 52.97850189
 52.9802119  52.98192191 52.98363192 52.98534193 52.98705194 52.98876195
 52.99047196 52.99218197 52.99389198 52.99560198 52.99731199 52.999022
 53.0

In [275]:
for i in range(50):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [276]:
# Den Oever
# <longitude>5.070195628786471</longitude>
# <latitude>52.93917167503315</latitude>

# Breezanddijk
# <longitude>5.197016484858931</longitude>
# <latitude>53.0229621352376</latitude>